In [2]:
import pandas as pd

In [4]:
mushrooms = pd.read_csv('./datasets/mushrooms.csv')

In [6]:
mushrooms.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
# get amount of unique values in each column
mushrooms.apply(lambda x: len(x.unique()))

class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64

In [ ]:
def build_graph(X, y, k=2):
  '''
  kDB algorithm

  Param:
  ----------------------
    
  Return:
  ----------------------
  graph edges
  '''
  #ensure data
  num_features = X.shape[1]
  x_nodes = list(range(num_features))
  y_node  = num_features

  #util func
  _x = lambda i:X[:,i]
  _x2comb = lambda i,j:(X[:,i], X[:,j])

  #feature indexes desc sort by mutual information
  sorted_feature_idxs = np.argsort([
    drv.information_mutual(_x(i), y) 
    for i in range(num_features)
  ])[::-1]

  #start building graph
  edges = []
  for iter, target_idx in tqdm(enumerate(sorted_feature_idxs)):
    target_node = x_nodes[target_idx]
    edges.append((y_node, target_node))

    parent_candidate_idxs = sorted_feature_idxs[:iter]
    if iter <= k:
      for idx in parent_candidate_idxs:
        edges.append((x_nodes[idx], target_node))
    else:
      first_k_parent_mi_idxs = np.argsort([
        drv.information_mutual_conditional(*_x2comb(i, target_idx), y)
        for i in parent_candidate_idxs
      ])[::-1][:k]
      first_k_parent_idxs = parent_candidate_idxs[first_k_parent_mi_idxs]

      for parent_idx in first_k_parent_idxs:
        edges.append((x_nodes[parent_idx], target_node))
  return edges

# Berechnung des Space für kdb

Allgemein ist die Berechnung für den Speicherbedarf wie folgt:

- mutual information: $n * m$ wobei $n,m$ die Ausprägungen der jeweiligen Variablen sind
- conditional mutual information: $(n * m) * l$ wobei l die Ausprägungen der dritten Variable ist

- KDB Algorithmus berechnet zuerst MI für jede Spalte zusammen mit Zielvariable und hält alle Ergebnisse im Speicher
  - Eine 2D-Matrix für jeden Kombination
  - Datentyp als int64 annehmen? (8 Bytes pro Element)

- Im zweiten Schritt werden je nach Höhe des $k$ die MIC berechnet
  - 3D-Raum mit den Dimensionen $(n * m) * l$?
  - Datentyp als in64 annehmen? (8 Bytes pro Element)

1. Berechnen mit Beispieldatensatz aus den drei Datensätzen
2. Berechnen für allgemeinen Datensätze

## Für Termin

- Tabelle mit Vergleich von Accuracy, SVD
- Tabelle mit Trainingsverlauf von Generator und Diskriminator
- Tabelle und Übersicht von den Berechnungen für Speicherverbrauch

dimensionality hoch skalieren
spalten =2, 4, 8, 16, 32, 64, 128, 256, 512, 1024
k = 1,2,3
10000 Datenpunkte

Zeit und Speicher nehmen
